In [4]:
import cv2
import time
from keras.preprocessing.image import img_to_array
import numpy as np
import os
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications.vgg16 import VGG16

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.models import Model

import tensorflow as tf
from keras.utils.vis_utils import plot_model

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

class acRec:
    def __init__(self):
        self.h=224
        self.w=224
        self.shp=(self.w,self.h)
    def arr(self,array):
        array=np.array(array)
        return array
    def vedioConvert(self,path):
        cap = cv2.VideoCapture(path)
        tic = time.perf_counter()
        i=0
        frames = np.zeros((50, self.h, self.w, 3), dtype=np.float)
        while cap.isOpened():
            _, frame = cap.read()
            try:
                RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            except:
                break
            img = cv2.resize(RGB,self.shp)
            img = np.expand_dims(img,axis=0)
            frames[i][:] = img
            if cv2.waitKey(1) == ord('q'):
                break
            i+=1
        toc = time.perf_counter()
        cap.release()
        cv2.destroyAllWindows()
        return [frames,(((path.split('/'))[-1]).split('_'))[0]]
    def drictor(self,path):
        dataTrain=np.zeros((126,50, self.h, self.w, 3), dtype=np.float)
        labelTrain=[]
        
        for dr in os.listdir(path):
            i=0
            for video in os.listdir(f'{path}/{dr}'):
                data=self.vedioConvert(f'{path}/{dr}/{video}')
                dataTrain[i][:]=data[0]
                labelTrain.append(data[1])
                i+=1
                    
        return self.arr(dataTrain),self.arr(labelTrain)
    
    def build_convnet(shape=(224, 224, 3)):
        momentum = .9
        model = keras.Sequential()
        model.add(layers.Conv2D(32, (3,3), input_shape=shape,
            padding='same', activation='relu'))
        model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization(momentum=momentum))

        model.add(layers.MaxPool2D())

        model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
        model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization(momentum=momentum))

        model.add(layers.MaxPool2D())

        model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
        model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization(momentum=momentum))

        # flatten...
        model.add(layers.GlobalMaxPool2D())
        return model
    def action_model(shape,classes):

        convnet = build_convnet(shape[1:])
        model = keras.Sequential()    
        model.add(layers.TimeDistributed(convnet, input_shape=shape))    
        model.add(layers.LSTM(30))    
        model.add(layers.Dense(1024, activation='relu'))
        model.add(layers.Dropout(.5))
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dropout(.5))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dropout(.5))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(classes, activation='softmax'))
        return model
    def define_model(self):
        classes=5

        shape=(sequence_length,self.h, self.w,3)
        modle=action_model(shape,classes)
        model.compile(
            optimizer="adam",
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        
    def define_model2(self):
        IMG_SIZE = 224
        embed_dim = 2048
#         sequence_length = 50
        dense_dim = 4
        num_heads = 1
        
        inputs = keras.Input(shape=(self.h, self.w,3))
        x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(inputs)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(x)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
        x = layers.MaxPool3D(pool_size=2)(x)
        x = layers.BatchNormalization()(x)

        x = layers.GlobalAveragePooling3D()(x)
        x = layers.Dense(units=512, activation="relu")(x)
        x = layers.Dropout(0.3)(x)
        outputs = layers.Dense(self.classes, activation="softmax")(x)
        model = Model(inputs, outputs)

        model.compile(
            optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
        )
        return model
    def define_model3(self):
        IMG_SIZE = 224
        embed_dim = 2048
        sequence_length = 50
        dense_dim = 4
        num_heads = 1
         
        inputs = keras.Input(shape=(self.h, self.w,3))
        x = PositionalEmbedding(
            sequence_length, embed_dim, name="frame_position_embedding"
        )(inputs)
        x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
        x = layers.GlobalMaxPooling1D()(x)
        x = layers.Dropout(0.5)(x)
        
        outputs = layers.Dense(self.classes, activation="softmax")(x)
        model = Model(inputs, outputs)

        model.compile(
            optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
        )
        return model
    
    
    def sict(self):
        inputs = keras.Input(shape=(self.h, self.w,3))
        
        model=layers.Conv2D(32,3,padding="same", activation="relu" )(inputs)
        model=layers.Conv2D(32,3,padding="same", activation="relu" )(inputs)
        
        model=layers.Conv2D(32,3,padding="same", activation="relu" )(inputs)
        model=layers.Conv2D(32,3,padding="same", activation="relu" )(inputs)
        model=layers.MaxPool2D()(model)
        
        model=layers.Conv2D(64, 3, padding="same", activation="relu")(model)        
        model=layers.Conv2D(64, 3, padding="same", activation="relu")(model)
        
        model=layers.Conv2D(64, 3, padding="same", activation="relu")(model)        
        model=layers.Conv2D(64, 3, padding="same", activation="relu")(model)
        model=layers.MaxPool2D()(model)
        model=layers.Dropout(0.5)(model)

        model=layers.Flatten()(model)
        model=layers.Dense(128,activation="relu")(model)
        model=layers.Dense(self.classes, activation="softmax")(model)
        model = Model(inputs, model)
        
        opt = SGD(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        
        opt = SGD(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    
    def vgg16(self):
        model = VGG16(include_top=False, input_shape=(224, 224, 3))
        # mark loaded layers as not trainable
        for layer in model.layers:
            layer.trainable = False
        # add new classifier layers
        flat1 = layers.Flatten()(model.layers[-1].output)
        class1 = layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
        output = layers.Dense(self.classes, activation='sigmoid')(class1)
        # define new model
        model = Model(inputs=model.inputs, outputs=output)
        # compile model
        opt = SGD(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    def simple(self):
        inputs = keras.Input(shape=(self.h, self.w,3))
        x = layers.Conv2D(filters=64, kernel_size=2, activation="relu")(inputs)
        x = layers.Conv2D(filters=64, kernel_size=2, activation="relu")(x)
        x = layers.MaxPool2D(pool_size=3)(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(filters=128, kernel_size=2, activation="relu")(x)
        x = layers.Conv2D(filters=128, kernel_size=2, activation="relu")(x)
        x = layers.MaxPool2D(pool_size=3)(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(filters=256, kernel_size=2, activation="relu")(x)
        x = layers.Conv2D(filters=256, kernel_size=2, activation="relu")(x)
        x = layers.MaxPool2D(pool_size=3)(x)
        x = layers.BatchNormalization()(x)
        
#         x = layers.Dense(units=3, activation="relu")(x)
        
        x = layers.Flatten()(x)
        x = layers.Dense(units=128, activation="relu")(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(self.classes, activation="softmax")(x)
        
        model = Model(inputs, x)
        
        opt = SGD(learning_rate=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        
        return  model
    
    def modelFit(self,model,train_it,test_it):
#         checkpoint = keras.callbacks.ModelCheckpoint(
#             'poseBasedAR_V1', save_weights_only=True, save_best_only=True, verbose=1
#             )
        
#         self.history = model.fit( [train_data], train_labels, validation_split=0.15, epochs=3, callbacks=[checkpoint],)
        self.history = model.fit(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=3, verbose=1)
        _, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
        print(f"Test accuracy: {round(acc * 100, 2)}%")
        model.save('AR_Skeleton_based_V1.h5')
        return model
        
        
    def summarize_diagnostics(history):
        plt.subplot(211)
        plt.title('Кросс энтропийн алдагдал Cross Entropy Loss')
        plt.plot(history.history['loss'], color='blue', label='train')
        plt.plot(history.history['val_loss'], color='orange', label='test')
        # plot accuracy
        plt.subplot(212)
        plt.title('Ангиллын нарийвчлал')
        plt.plot(history.history['accuracy'], color='blue', label='train')
        plt.plot(history.history['val_accuracy'], color='orange', label='test')
        # save plot to file
        filename = sys.argv[0].split('/')[-1]
        plt.savefig(filename + '_plot.png')
        plt.show()
        plt.close()

    def process(self,path):
#         dataTrain,labelTrain=self.drictor(path)
        datagen = ImageDataGenerator()
        train_it=datagen.flow_from_directory(path+'/train',class_mode='categorical', batch_size=64, target_size=(224, 224))
        className=[]
        for i in train_it.class_indices:
            className.append(i)
        self.classNameIndex=className
        test_it=datagen.flow_from_directory(path+'/test',class_mode='categorical', batch_size=8, target_size=(224, 224))
        print(train_it.num_classes)
        self.classes=train_it.num_classes
        
#         model=self.sict()

#         model=self.simple()
        model=self.vgg16()
        model.summary() 
        plot_model(model, to_file='model_plot.png', show_shapes=False, show_layer_names=True)
        model=self.modelFit(model,train_it,test_it)
#         summarize_diagnostics(self.history)
        return model
    

In [5]:
ac=acRec()
model=ac.process('testdata/dataset/train2')
model.save('AR_Skeleton_based_signal_RGB_V1.h5')
ac.classNameIndex

Found 9037 images belonging to 7 classes.
Found 2810 images belonging to 7 classes.
7
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                       

2022-05-25 00:23:15.209183: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.
2022-05-25 00:23:15.482571: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.
2022-05-25 00:23:16.310177: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 205520896 exceeds 10% of free system memory.
2022-05-25 00:23:16.438750: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.
2022-05-25 00:23:16.811348: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 411041792 exceeds 10% of free system memory.


142/142 [==============================] - 4099s 29s/step - loss: 0.6939 - accuracy: 0.9475 - val_loss: 0.1319 - val_accuracy: 0.9754
Epoch 2/3
142/142 [==============================] - 1412s 10s/step - loss: 0.0024 - accuracy: 0.9986 - val_loss: 0.0601 - val_accuracy: 0.9875
Epoch 3/3
142/142 [==============================] - 1369s 10s/step - loss: 8.7031e-04 - accuracy: 0.9997 - val_loss: 0.0712 - val_accuracy: 0.9875
Test accuracy: 98.75%


['2_Degee', 'BARUUN', 'Deeshee', 'ZVVN', 'dooshoo', 'duussan', 'zogson']

In [2]:
ac=acRec()
model=ac.process('testdata/dataset/pSignal')
model.save('AR_Skeleton_based_signal_V1.h5')

Found 10637 images belonging to 7 classes.
Found 1210 images belonging to 7 classes.
7


2022-05-23 10:04:36.845865: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dashnyam/anaconda3/envs/ds/lib/python3.10/site-packages/cv2/../../lib64:
2022-05-23 10:04:36.846018: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-23 10:04:36.846084: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dashnyam-Latitude-5520): /proc/driver/nvidia/version does not exist
2022-05-23 10:04:36.847671: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fl

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2022-05-23 10:04:38.277483: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.
2022-05-23 10:04:38.563041: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.


  1/167 [..............................] - ETA: 20:01 - loss: 8.3779 - accuracy: 0.1562

2022-05-23 10:04:44.793311: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.
2022-05-23 10:04:45.058410: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.


  2/167 [..............................] - ETA: 19:30 - loss: 7.9619 - accuracy: 0.2109

2022-05-23 10:04:51.886685: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 822083584 exceeds 10% of free system memory.


167/167 [==============================] - 1448s 9s/step - loss: 0.3876 - accuracy: 0.9662 - val_loss: 0.1169 - val_accuracy: 0.9736
Epoch 2/3
167/167 [==============================] - 1451s 9s/step - loss: 0.0046 - accuracy: 0.9984 - val_loss: 0.1280 - val_accuracy: 0.9727
Epoch 3/3
167/167 [==============================] - 1420s 9s/step - loss: 2.9460e-04 - accuracy: 0.9999 - val_loss: 0.0768 - val_accuracy: 0.9752
Test accuracy: 97.52%


In [91]:
ac=acRec()
model=ac.process('testdata/dataset/trained')
model.save('ounPoseArModel_V1.h5')

Found 718 images belonging to 5 classes.
Found 71 images belonging to 5 classes.
5
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_221 (Conv2D)         (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_90 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_222 (Conv2D)         (None, 112, 112, 64)      18496     
                                                                 
 conv2d_223 (Conv2D)         (None, 112, 112, 64)      36928     
                                                                 
 conv2d_224 (Conv2D)         (None, 112, 

In [87]:
!pip install pydot

  Using cached pydot-1.4.2-py2.py3-none-any.whl (21 kB)


In [61]:
ac=acRec()
model=ac.process('testdata/dataset/trained')
model.save('ounPoseArModel_V1.h5')

Found 718 images belonging to 5 classes.
Found 71 images belonging to 5 classes.
5
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_138 (Conv2D)         (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_67 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_139 (Conv2D)         (None, 112, 112, 64)      18496     
                                                                 
 conv2d_140 (Conv2D)         (None, 112, 112, 64)      36928     
                                                                 
 conv2d_141 (Conv2D)         (None, 112, 1

In [7]:
ac=acRec()
model=ac.process('testdata/dataset/TRAINRGB')
model.save('AR_RGB_based_V1.h5')

Found 294 images belonging to 5 classes.
Found 130 images belonging to 5 classes.
5
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                         

In [76]:
ac=acRec()
model=ac.process('testdata/dataset/trained')
model.save('AR_RGB_based_V1.h5')

Found 718 images belonging to 5 classes.
Found 71 images belonging to 5 classes.
5
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                          

In [69]:
ac.classNameIndex

['алхаж байна', 'даллаж байна', 'зогсож байна', 'сунайж байна', 'сууж байна']

In [73]:
def summarize_diagnostics(history):
    plt.subplot(211)
    plt.title('Кросс энтропийн алдагдал Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    plt.subplot(212)
    plt.title('Ангиллын нарийвчлал')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    plt.savefig('_plot.png')
    plt.close()

In [78]:
def process(filename):
   
    outPath=f'./actRec/out/{datetime.today().strftime("%Y%m%d")}'
    imgPath=f'./actRec/img/{datetime.today().strftime("%Y%m%d")}'

    nowDate=datetime.now().strftime(" %T%m%S")
    try:
        os.mkdir(outPath)
    except :
        print(f'{outPath} already created')
    try:
        os.mkdir(imgPath)
    except :
        print(f'{imgPath} already created')

    h=280
    w=432
    shp=(w,h)
    cap = cv2.VideoCapture(f'../static/uploads/{filename}')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    outVideoPath=f'{outPath}/{nowDate}.mp4'
    outImgPath=f'{imgPath}/{nowDate}.jpg'
    out = cv2.VideoWriter(outVideoPath, fourcc, 25.0, shp)
    i=0
    endPoint=0
    tic = time.perf_counter()   
    saveImg=False
    Action='...'
    Accuracy=0


    while cap.isOpened():
        _, frame = cap.read()
        try:
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        except:
            break
        
        results = pose.process(RGB)
        blank_image = np.zeros(RGB.shape, np.uint8)
        
        mp_drawing.draw_landmarks(
        blank_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=draw,
        connection_drawing_spec=drawline
        )
        
        if 20>=i:
            img=cv2.resize(blank_image,(224, 224))
            img = img_to_array(img)
            img = img.reshape(1, 224, 224, 3)
            img = img.astype('float32')
            result = model.predict(img)
            max=getMax(result[0])
            Action = classes[max[1]]
            Accuracy = max[0]
            if not saveImg:
                cv2.putText(blank_image, f'{Action} accuracy = ({Accuracy})', (10,10), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 0), 8, cv2.LINE_AA)
                cv2.imwrite(outImgPath,blank_image)
                saveImg = True
            i=0
        i+=1
        endPoint+=1
        x=blank_image.shape[0]*0.32
        y=blank_image.shape[1]*0.05
        cv2.putText(blank_image, f'{Action} accuracy = ({Accuracy})', (int(x),int(y)), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 8, cv2.LINE_AA)
        blank_image = cv2.resize(blank_image,shp)
        out.write(blank_image)        
        if(endPoint>=25*22):
            break
        if cv2.waitKey(1) == ord('q'):
            break
    toc = time.perf_counter()
    print(f"timer : \n\t\t{toc - tic:0.4f} seconds")
    out.release()
    cap.release()
    cv2.destroyAllWindows() 
    return [outVideoPath,outImgPath]

In [118]:
blank_image = cv2.imread('./actRec/img/20220427/ 12:38:380438.jpg')

blank_image = cv2.resize(blank_image,(432,280))
x=blank_image.shape[0]*0.32
y=blank_image.shape[1]*0.05
cv2.putText(blank_image, f'asas accuracy = ({round(0.9898989898 * 100, 2)})', (int(x),int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.63, (0, 255, 0), 0, cv2.LINE_AA)
cv2.imwrite('./actRec/img/20220427/ TEST.jpg',blank_image)        


True

In [45]:
import cv2
import mediapipe as mp
import time
import os
from datetime import datetime
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
draw=mp_drawing.DrawingSpec((12, 24, 224), 6, 13)
drawline=mp_drawing.DrawingSpec((224, 224, 224), 45, 55)

def dataProcess(filename,name):
    outPath=f'./testdata/out/{datetime.today().strftime("%Y%m%d")}'
    imgPath=f'./testdata/img/{datetime.today().strftime("%Y%m%d")}'
    imgPathPose=f'./testdata/imgPose/{datetime.today().strftime("%Y%m%d")}'
    # nowDate=datetime.now()
    nowDate=datetime.now().strftime(" %T%m%S")
    try:
        os.mkdir(outPath)
    except :
        print(f'{outPath} already created')
    try:
        os.mkdir(imgPath)
    except :
        print(f'{imgPath} already created')
    try:
        os.mkdir(imgPathPose)
    except :
        print(f'{imgPathPose} already created')
    # create capture object
    h=280
    w=432
    shp=(w,h)
    cap = cv2.VideoCapture(f'../static/uploads/{filename}')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(f'{outPath}/{name} {nowDate}.mp4', fourcc, 25.0, shp)
    i=0
    endPoint=0
    cutCounter=0
    tic = time.perf_counter()
    while cap.isOpened():
        _, frame = cap.read()
        try:
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        except:
            break
        
        results = pose.process(RGB)
        
        blank_image = np.zeros(RGB.shape, np.uint8)

        mp_drawing.draw_landmarks(
        blank_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=draw,
        connection_drawing_spec=drawline
        )

        i+=1
        if i==25*2:
            out.release()
            out = cv2.VideoWriter(f'{outPath}/{name}{nowDate}_({cutCounter}).mp4', fourcc, 25.0, shp)
            cv2.imwrite(f'{imgPath}/{name}{nowDate}_({cutCounter}).jpg',frame)
            cv2.imwrite(f'{imgPathPose}/{name}{nowDate}_{cutCounter})(1).jpg',blank_image)
        
            cutCounter+=1
            endPoint+=i
            i=0            
        elif i==25:
            cv2.imwrite(f'{imgPathPose}/{name}{nowDate}_({cutCounter})(0).jpg',blank_image)
        if(endPoint>=25*3*60):
            break
        blank_image = cv2.resize(blank_image,shp)
        out.write(blank_image)
    
        if cv2.waitKey(1) == ord('q'):
            break
    toc = time.perf_counter()
    print(f"timer : \n\t\t{toc - tic:0.4f} seconds")
    out.release()
    cap.release()
    cv2.destroyAllWindows() 


    
a='PUSH_UP'
b=2
dataProcess('PUSH_UP_H2.mp4',f'{a}_H{b}')

./testdata/out/20220427 already created
./testdata/img/20220427 already created
./testdata/imgPose/20220427 already created
timer : 
		52.7155 seconds


In [4]:
import cv2
from datetime import datetime
from Ds5sD import PoseBased

In [9]:
cap=cv2.VideoCapture(0)
aaa=''
t=True
pd=PoseBased()
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
tf=True
i,save=0,0
while True:
    ret,frame = cap.read()
    if not ret:  
        continue  
    res=pd.process(frame)
    frame = cv2.hconcat([frame, res[0]])
    i+=1
    if tf:
        outVideoPath=f'./pose/out/{datetime.today()}.mp4'
        out = cv2.VideoWriter(outVideoPath, fourcc, 25.0, (frame.shape[1],frame.shape[0]))
        out.write(frame)
        tf=False
        save=0
    else:
        save+=1
        out.write(frame)
    if save==250:
        tf=True
        out.release()
    if i==600:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

timer :	0.1364 seconds ==> wave
timer :	0.1210 seconds ==> wave
timer :	0.0874 seconds ==> wave
timer :	0.1059 seconds ==> wave
timer :	0.1073 seconds ==> wave
timer :	0.0968 seconds ==> wave
timer :	0.0940 seconds ==> wave
timer :	0.0899 seconds ==> walk
timer :	0.1318 seconds ==> walk
timer :	0.0884 seconds ==> walk
timer :	0.1042 seconds ==> walk
timer :	0.1035 seconds ==> walk
timer :	0.0963 seconds ==> walk
timer :	0.1051 seconds ==> walk
timer :	0.1099 seconds ==> walk
timer :	0.1035 seconds ==> walk
timer :	0.1017 seconds ==> walk
timer :	0.1176 seconds ==> walk
timer :	0.1213 seconds ==> walk
timer :	0.1327 seconds ==> walk
timer :	0.1269 seconds ==> walk
timer :	0.1271 seconds ==> walk
timer :	0.0876 seconds ==> walk
timer :	0.1382 seconds ==> walk
timer :	0.1463 seconds ==> walk
timer :	0.0968 seconds ==> walk
timer :	0.0885 seconds ==> walk
timer :	0.1045 seconds ==> walk
timer :	0.1138 seconds ==> walk
timer :	0.0971 seconds ==> walk
timer :	0.0862 seconds ==> walk
timer :	

timer :	0.1316 seconds ==> wave
timer :	0.0910 seconds ==> stand
timer :	0.0967 seconds ==> stand
timer :	0.0966 seconds ==> walk
timer :	0.0859 seconds ==> walk
timer :	0.1076 seconds ==> walk
timer :	0.1034 seconds ==> walk
timer :	0.1413 seconds ==> walk
timer :	0.1109 seconds ==> walk
timer :	0.1055 seconds ==> walk
timer :	0.1134 seconds ==> walk
timer :	0.1123 seconds ==> walk
timer :	0.1026 seconds ==> wave
timer :	0.1108 seconds ==> wave
timer :	0.1278 seconds ==> wave
timer :	0.1070 seconds ==> stand
timer :	0.1125 seconds ==> stand
timer :	0.1359 seconds ==> wave
timer :	0.1259 seconds ==> stand
timer :	0.1049 seconds ==> stand
timer :	0.0922 seconds ==> stand
timer :	0.1066 seconds ==> stand
timer :	0.1420 seconds ==> stand
timer :	0.1299 seconds ==> stand
timer :	0.1398 seconds ==> stand
timer :	0.1256 seconds ==> stand
timer :	0.1008 seconds ==> stand
timer :	0.1225 seconds ==> stand
timer :	0.1413 seconds ==> stand
timer :	0.1400 seconds ==> stand
timer :	0.1061 seconds =

timer :	0.0914 seconds ==> walk
timer :	0.1086 seconds ==> walk
timer :	0.1374 seconds ==> walk
timer :	0.0964 seconds ==> walk
timer :	0.1017 seconds ==> walk
timer :	0.1066 seconds ==> walk
timer :	0.1005 seconds ==> sit
timer :	0.0857 seconds ==> sit
timer :	0.1309 seconds ==> sit
timer :	0.1322 seconds ==> sit
timer :	0.1409 seconds ==> sit
timer :	0.0902 seconds ==> sit
timer :	0.1028 seconds ==> sit
timer :	0.1312 seconds ==> sit
timer :	0.1098 seconds ==> sit
timer :	0.1418 seconds ==> sit
timer :	0.1337 seconds ==> sit
timer :	0.1281 seconds ==> sit
timer :	0.1597 seconds ==> sit
timer :	0.1534 seconds ==> sit
timer :	0.1298 seconds ==> sit
timer :	0.1199 seconds ==> sit
timer :	0.1271 seconds ==> sit
timer :	0.1128 seconds ==> sit
timer :	0.1009 seconds ==> sit
timer :	0.0969 seconds ==> sit
timer :	0.0980 seconds ==> sit
timer :	0.1040 seconds ==> sit
timer :	0.1316 seconds ==> sit
timer :	0.1252 seconds ==> sit
timer :	0.1449 seconds ==> sit
timer :	0.1242 seconds ==> sit
ti

In [2]:
import cv2
import mediapipe as mp
import time
import os
from datetime import datetime
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
draw=mp_drawing.DrawingSpec((12, 24, 224), 2, 10)
drawline=mp_drawing.DrawingSpec((224, 224, 224),25, 35)


videoPath="./testdata/dataset/SIGNAL"
outPath="./testdata/dataset/Skeleton"

for i in os.listdir(videoPath):
    actor=1
    outp=(outPath+"/"+i)
    try:
        os.mkdir(outp)
    except :
        ""
    for i2 in os.listdir(videoPath+"/"+i):
#         print(videoPath+"/"+i+"/"+i2+"/"+str(actor))
        outDataP=outp+"/"+i+"_"+str(actor)
        cap=cv2.VideoCapture(videoPath+"/"+i+"/"+i2)
        count=1
        while True:
            ret,frame = cap.read()
            if not ret:
                break
            try:
                RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            except:
                break
            results=pose.process(RGB)
#             blank_image = np.zeros(RGB.shape, np.uint8)

            mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=draw,
            connection_drawing_spec=drawline
            )
            cv2.imwrite(f'{outDataP}+{count}.jpg',frame)
            count+=1
        cap.release()    
        actor+=1
for i in os.listdir("./testdata/dataset/SIGNAL"):
    actor=1
    outp=("./testdata/dataset/Test"+"/"+i)
    try:
        os.mkdir(outp)
    except :
        ""

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
